In [12]:
!pip install pandas plotly numpy

In [13]:
import pandas as pd
import numpy as np

df0 = pd.read_csv('data/0_preprocessed.csv')
df0.head(2)

,timestamp,duration,song,artist,album,song_artist
0,7/5/2019 3:00,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,7/5/2019 3:01,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker


In [14]:
# Get albums with 5 or more distinct songs
albums = df0.groupby(['album', 'artist'])['song'].nunique().reset_index()
albums = albums[albums['song'] >= 5]

In [15]:
#Get album duration
album_duration = df0.groupby(['album', 'artist'])['duration'].sum().reset_index()

In [16]:
df4_albums = pd.merge(albums, album_duration, on=['album', 'artist'])
df4_albums = df4_albums.rename(columns={
    'album': 'child',
    'artist': 'parent',
    'song': 'n_songs'})
df4_albums['duration'] = df4_albums['duration'].astype(int)
df4_albums = df4_albums.sort_values('duration', ascending=False)
df4_albums = df4_albums[df4_albums['child'] != df4_albums['parent']]
df4_albums = df4_albums.reset_index(drop=True)
df4_albums.head(10)

,child,parent,n_songs,duration
0,Speak Now,Taylor Swift,20,26647
1,Red,Taylor Swift,21,24946
2,Una Mattina,Ludovico Einaudi,13,12452
3,Fearless - Big Machine Radio Release Special,Taylor Swift,15,8650
4,m-tp M-TP,Sơn Tùng M-TP,7,7118
5,Divenire,Ludovico Einaudi,10,4650
6,Fearless - Platinum Edition,Taylor Swift,11,3074
7,In A Time Lapse,Ludovico Einaudi,7,2562
8,Born To Die - The Paradise Edition,Lana Del Rey,6,1136
9,1989,Taylor Swift,16,838


In [17]:
fearless = df4_albums[df4_albums['child'].str.contains('Fearless')]
df4_albums = df4_albums[~df4_albums['child'].str.contains('Fearless')]

new_row = pd.DataFrame({
    'child': ['Fearless'],
    'parent': ['Taylor Swift'],
    'n_songs': [fearless.n_songs.sum()],
    'duration': [fearless.duration.sum()]
})
df4_albums = pd.concat([df4_albums, new_row]).sort_values('duration', ascending=False).reset_index(drop=True)
df4_albums

,child,parent,n_songs,duration
0,Speak Now,Taylor Swift,20,26647
1,Red,Taylor Swift,21,24946
2,Una Mattina,Ludovico Einaudi,13,12452
3,Fearless,Taylor Swift,26,11724
4,m-tp M-TP,Sơn Tùng M-TP,7,7118
5,Divenire,Ludovico Einaudi,10,4650
6,In A Time Lapse,Ludovico Einaudi,7,2562
7,Born To Die - The Paradise Edition,Lana Del Rey,6,1136
8,1989,Taylor Swift,16,838
9,Different World,Alan Walker,5,450


In [18]:
df4_artists = df4_albums.groupby('parent').agg({'n_songs': 'sum', 'duration': 'sum'}).reset_index()
df4_artists.rename(columns={'parent': 'child'}, inplace=True)
df4_artists.insert(2, 'parent', 'Top Albums')
new_row = pd.DataFrame({
    'parent': [''],
    'child': ['Top Albums'],
    'n_songs': [df4_artists.n_songs.sum()],
    'duration': [int(df4_artists.duration.sum())]
})
df4_artists = pd.concat([new_row, df4_artists], ignore_index=True)
df4_artists

,parent,child,n_songs,duration
0,,Top Albums,279,94836
1,Top Albums,AMEE,5,14
2,Top Albums,Alan Walker,5,450
3,Top Albums,Ariana Grande,23,324
4,Top Albums,Christina Perri,5,59
5,Top Albums,Ed Sheeran,5,145
6,Top Albums,Katy Perry,5,247
7,Top Albums,Lana Del Rey,11,1216
8,Top Albums,Ludovico Einaudi,59,20144
9,Top Albums,Post Malone,15,269


In [19]:
df4 = pd.concat([df4_artists, df4_albums], ignore_index=True)
df4['scale'] = np.log10(df4['duration']).astype(int)
df4.loc[0, 'scale'] = 0
df4

,parent,child,n_songs,duration,scale
0,,Top Albums,279,94836,0
1,Top Albums,AMEE,5,14,1
2,Top Albums,Alan Walker,5,450,2
3,Top Albums,Ariana Grande,23,324,2
4,Top Albums,Christina Perri,5,59,1
5,Top Albums,Ed Sheeran,5,145,2
6,Top Albums,Katy Perry,5,247,2
7,Top Albums,Lana Del Rey,11,1216,3
8,Top Albums,Ludovico Einaudi,59,20144,4
9,Top Albums,Post Malone,15,269,2


In [20]:
import plotly.express as px

fig = px.sunburst(
    df4,
    names='child',
    parents='parent',
    values='scale',
    hover_data={
        'child': False,
        'parent': False,
        'n_songs': True,
        'duration': True,
        'scale': False
    },
)
fig.show()

In [21]:
df4.to_csv('4_sunburst_albums.csv', index=False)